### Imports

In [ ]:
# Imports
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange #pip install einops
from typing import List
import random
import math
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from timm.utils import ModelEmaV3 #pip install timm
from tqdm import tqdm #pip install tqdm
import matplotlib.pyplot as plt #pip install matplotlib
import torch.optim as optim
import numpy as np
import nibabel as nib

### Early Stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=1e-4, model_path="best_model.pth"):
        self.patience = patience
        self.delta = delta  # Minimum improvement threshold
        self.best_score = None
        self.counter = 0
        self.model_path = model_path  # Save the best model

    def step(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.save_checkpoint(model)
            return False

        elif val_loss < self.best_score - self.delta:  # Require significant improvement
            self.best_score = val_loss
            self.counter = 0
            self.save_checkpoint(model)  # Save best model
            return False

        else:
            self.counter += 1
            if self.counter >= self.patience:
                print("Early Stopping Triggered.")
                return True  # Stop training
            return False

    def save_checkpoint(self, model):
        """Save the model when validation loss improves."""
        torch.save(model.state_dict(), self.model_path)
        print(f"Model saved with val_loss: {self.best_score:.6f}")

### Save Checkpoint

In [ ]:
def save_checkpoint(model, optimizer, ema, filename, folder='~/Siddesh/checkpoints'):
    os.makedirs(folder, exist_ok=True)
    checkpoint_path = os.path.join(folder, filename)

    checkpoint = {
        'weights': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'ema': ema.state_dict()
    }

    torch.save(checkpoint, checkpoint_path)
    print(f"[INFO] Checkpoint saved at: {checkpoint_path}")

### Data Loader

In [ ]:
class BRATSDataset(torch.utils.data.Dataset):
    def __init__(self, directory, test_flag = False, resize = False):
        super().__init__()
        self.directory = os.path.expanduser(directory)
        self.test_flag = test_flag
        self.seqtypes = ['t1', 't1ce', 't2', 'flair']  # Only these 4 modalities
        self.seqtypes_set = set(self.seqtypes)
        self.database = []

        for root, dirs, files in os.walk(self.directory):
            if not dirs:  # Leaf directory
                datapoint = {}
                for f in sorted(files):
                    seqtype = f.split('_')[3] if len(f.split('_')) > 3 else None
                    if seqtype in self.seqtypes_set:
                        datapoint[seqtype] = os.path.join(root, f)

                # Ensure only the 4 required modalities are present
                if set(datapoint.keys()) == self.seqtypes_set:
                    self.database.append(datapoint)

    def __getitem__(self, index):
        filedict = self.database[index]
        images = []

        for seqtype in self.seqtypes:
            # Load the image
            img = nib.load(filedict[seqtype]).get_fdata()
            img = torch.tensor(img, dtype=torch.float32)  # Convert to tensor

            if self.resize:
                # Resize to 128x128 (assuming input is [H, W])
                img = F.interpolate(img.unsqueeze(0).unsqueeze(0), size=(128, 128), mode="bilinear", align_corners=False)
                img = img.squeeze()  # Remove extra dimensions

            images.append(img)

        # Stack all modalities (4, H, W)
        images = torch.stack(images)

        return images

    def __len__(self):
        return len(self.database)

### Timestep embedding

In [24]:
class SinusoidalEmbeddings(nn.Module):
    def __init__(self, time_steps: int, embed_dim: int):
        super().__init__()
        position = torch.arange(time_steps, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float32) * -(math.log(10000.0) / embed_dim))
        embeddings = torch.zeros(time_steps, embed_dim, requires_grad=False)
        embeddings[:, 0::2] = torch.sin(position * div)
        embeddings[:, 1::2] = torch.cos(position * div)
        self.register_buffer('embeddings', embeddings)

    def forward(self, x, t):
        return self.embeddings[t].to(x.device)[:, :, None, None]

### Residual Blocks

In [25]:
# Residual Blocks
class ResBlock(nn.Module):
    def __init__(self, C: int, num_groups: int, dropout_prob: float):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.gnorm1 = nn.GroupNorm(num_groups=num_groups, num_channels=C)
        self.gnorm2 = nn.GroupNorm(num_groups=num_groups, num_channels=C)
        self.conv1 = nn.Conv2d(C, C, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(C, C, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x, embeddings):
        x = x + embeddings[:, :x.shape[1], :, :]
        r = self.conv1(self.relu(self.gnorm1(x)))
        r = self.dropout(r)
        r = self.conv2(self.relu(self.gnorm2(r)))
        return r + x

### Self Attention

In [26]:
class Attention(nn.Module):
    def __init__(self, C: int, num_heads: int, dropout_prob: float):
        super().__init__()
        self.num_heads = num_heads
        self.proj1 = nn.Linear(C, C * 3)
        self.proj2 = nn.Linear(C, C)
        self.dropout_prob = dropout_prob

    def forward(self, x):
        h, w = x.shape[2:]
        x = rearrange(x, 'b c h w -> b (h w) c')
        x = self.proj1(x)
        x = rearrange(x, 'b L (C H K) -> K b H L C', K=3, H=self.num_heads)
        q, k, v = x[0], x[1], x[2]
        x = F.scaled_dot_product_attention(q, k, v, is_causal=False, dropout_p=self.dropout_prob)
        x = rearrange(x, 'b H (h w) C -> b h w (C H)', h=h, w=w)
        x = self.proj2(x)
        return rearrange(x, 'b h w C -> b C h w')

### UNet

In [ ]:
class UnetLayer(nn.Module):
    def __init__(self,
            upscale: bool,
            attention: bool,
            num_groups: int,
            dropout_prob: float,
            num_heads: int,
            C: int):
        super().__init__()
        self.ResBlock1 = ResBlock(C, num_groups, dropout_prob)
        self.ResBlock2 = ResBlock(C, num_groups, dropout_prob)
        if upscale:
            self.conv = nn.ConvTranspose2d(C, C//2, kernel_size=4, stride=2, padding=1)
        else:
            self.conv = nn.Conv2d(C, C*2, kernel_size=3, stride=2, padding=1)
        self.attention_layer = Attention(C, num_heads, dropout_prob) if attention else None

    def forward(self, x, embeddings):
        x = self.ResBlock1(x, embeddings)
        if self.attention_layer:
            x = self.attention_layer(x)
        x = self.ResBlock2(x, embeddings)
        return self.conv(x), x

class UNET(nn.Module):
    def __init__(self,
            Channels: List = [64, 128, 256, 512, 512, 384],
            Attentions: List = [False, True, False, False, False, True],
            Upscales: List = [False, False, False, True, True, True],
            num_groups: int = 32,
            dropout_prob: float = 0.1,
            num_heads: int = 2,
            input_channels: int = 4,
            output_channels: int = 4,
            time_steps: int = 500):

        super().__init__()
        self.num_layers = len(Channels)
        self.shallow_conv = nn.Conv2d(input_channels, Channels[0], kernel_size=3, padding=1)
        self.embeddings = SinusoidalEmbeddings(time_steps=time_steps, embed_dim=max(Channels))

        out_channels = (Channels[-1] // 2) + Channels[0]
        self.late_conv = nn.Conv2d(out_channels, out_channels // 2, kernel_size=3, padding=1)
        self.output_conv = nn.Conv2d(out_channels // 2, output_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

        # Initialize UNet layers
        for i in range(self.num_layers):
            layer = UnetLayer(
                upscale=Upscales[i],
                attention=Attentions[i],
                num_groups=num_groups,
                dropout_prob=dropout_prob,
                C=Channels[i],
                num_heads=num_heads
            )
            setattr(self, f'Layer{i+1}', layer)

    def forward(self, x, t):
        x = self.shallow_conv(x)
        embeddings = self.embeddings(x, t)
        residuals = []

        for i in range(self.num_layers // 2):
            layer = getattr(self, f'Layer{i+1}')
            x, res = layer(x, embeddings)
            residuals.append(res)

        for i in range(self.num_layers//2, self.num_layers):
            layer = getattr(self, f'Layer{i+1}')
            x = torch.concat((layer(x, embeddings)[0], residuals[self.num_layers-i-1]), dim=1)
        return self.output_conv(self.relu(self.late_conv(x)))

### Time Step Scheduler

In [28]:
class DDPM_Scheduler(nn.Module):
    def __init__(self, num_time_steps=1000):
        super().__init__()
        beta = torch.linspace(1e-4, 0.02, num_time_steps)
        alpha = 1 - beta
        self.register_buffer('alpha', torch.cumprod(alpha, dim=0))

    def forward(self, t):
        return self.alpha[t]

In [29]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

### Training

In [ ]:
# Debug logging helper
def print_log(message):
    print(f"[Log] {message}")

# Normalize images to [-1, 1]
def normalize_images(images):
    return (images - 0.5) / 0.5

# Set device (CPU/GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print_log(f"Using device: {device}")

def train(batch_size: int=16,
          num_time_steps: int=500,
          num_epochs: int=15,
          seed: int=-1,
          ema_decay: float=0.9999,
          lr=2e-5,
          data_dir: str='/healthy',
          checkpoint_path: str=None):

    # Set random seed
    seed = random.randint(0, 2**32-1) if seed == -1 else seed
    print_log(f"Random seed set to: {seed}")
    set_seed(seed)

    # Load dataset
    print_log("Loading dataset...")

    train_dataset = BRATSDataset(directory=data_dir)
    print_log(f"Dataset length: {len(train_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0, pin_memory=True)

    try:
        sample_batch = next(iter(train_loader))
        print_log(f"First batch shape: {sample_batch.shape}")
    except Exception as e:
        print(f"[ERROR] DataLoader issue: {e}")
        return

    _, H, W = sample_batch.shape[1:]

    # Initialize model
    scheduler = DDPM_Scheduler(num_time_steps=num_time_steps)
    model = UNET(
        input_channels=4,
        output_channels=4,
        Channels=[64, 128, 256, 512, 512, 384] if max(H, W) <= 256 else [32, 64, 128, 256, 256, 192]
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    ema = ModelEmaV3(model, decay=ema_decay)

    # Load checkpoint
    if checkpoint_path is not None and os.path.exists(checkpoint_path):
        print_log("Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path, map_location="cpu")
        model.load_state_dict(checkpoint['weights'])
        ema.load_state_dict(checkpoint['ema'])
        optimizer.load_state_dict(checkpoint['optimizer'])

    criterion = nn.MSELoss(reduction='mean')

    try:
        print_log(f"Starting training for {num_epochs} epochs...")

        for i in range(num_epochs):
            print_log(f"Starting epoch {i+1}/{num_epochs}...")
            total_loss = 0

            for bidx, x in enumerate(tqdm(train_loader, desc=f"----------------------------------------------------------------\nEpoch {i+1}/{num_epochs}\n")):
                print("")
                print_log(f"Processing batch {bidx+1}/{len(train_loader)}...")

                x = x.to(device)
                print_log(f"Batch shape: {x.shape}")

                # Normalize to [-1, 1]
                x = normalize_images(x)

                # Resize if necessary
                if max(H, W) > 256:
                    x = F.interpolate(x, size=(256, 256), mode='bilinear', align_corners=False)

                t = torch.randint(0, num_time_steps, (x.shape[0],)).to(device)
                e = torch.randn_like(x, requires_grad=False).to(device)
                a = scheduler.alpha[t].view(x.shape[0], 1, 1, 1).to(device)

                x = (torch.sqrt(a) * x) + (torch.sqrt(1 - a) * e)

                # Apply diffusion
                output = model(x, t)
                optimizer.zero_grad()
                loss = criterion(output, e)
                total_loss += loss.item()
                print_log(f"Loss: {loss.item()}")

                loss.backward()
                optimizer.step()
                ema.update(model)

            print_log(f"Epoch {i+1} | Loss {total_loss / len(train_loader):.5f}")

            # Save checkpoint every 5 epochs
            if (i + 1) % 5 == 0:
                save_checkpoint(model, optimizer, ema, f'brats_ddpm_checkpoint_epoch_{i+1}.pth')

    except KeyboardInterrupt:
        print("\n[WARNING] Training interrupted. Saving last checkpoint...")
        save_checkpoint(model, optimizer, ema, 'brats_ddpm_interrupted_checkpoint.pth')

    print_log("Training complete.")
    save_checkpoint(model, optimizer, ema, 'brats_ddpm_final_checkpoint.pth')


[Log] Using device: cpu


### Inference

In [15]:
def display_reverse(images: List):
    fig, axes = plt.subplots(1, 10, figsize=(10,1))
    for i, ax in enumerate(axes.flat):
        x = images[i].squeeze(0)
        x = rearrange(x, 'c h w -> h w c')
        x = x.numpy()
        ax.imshow(x)
        ax.axis('off')
    plt.show()

def inference(checkpoint_path: str=None,
              num_time_steps: int=1000,
              ema_decay: float=0.9999, ):
    checkpoint = torch.load(checkpoint_path)
    model = UNET().cpu()
    model.load_state_dict(checkpoint['weights'])
    ema = ModelEmaV3(model, decay=ema_decay)
    ema.load_state_dict(checkpoint['ema'])
    scheduler = DDPM_Scheduler(num_time_steps=num_time_steps)
    times = [0,15,50,100,200,300,400,550,700,999]
    images = []

    with torch.no_grad():
        model = ema.module.eval()
        for i in range(10):
            z = torch.randn(1, 1, 32, 32)
            for t in reversed(range(1, num_time_steps)):
                t = [t]
                temp = (scheduler.beta[t]/( (torch.sqrt(1-scheduler.alpha[t]))*(torch.sqrt(1-scheduler.beta[t])) ))
                z = (1/(torch.sqrt(1-scheduler.beta[t])))*z - (temp*model(z.cpu(),t).cpu())
                if t[0] in times:
                    images.append(z)
                e = torch.randn(1, 1, 32, 32)
                z = z + (e*torch.sqrt(scheduler.beta[t]))
            temp = scheduler.beta[0]/( (torch.sqrt(1-scheduler.alpha[0]))*(torch.sqrt(1-scheduler.beta[0])) )
            x = (1/(torch.sqrt(1-scheduler.beta[0])))*z - (temp*model(z.cpu(),[0]).cpu())

            images.append(x)
            x = rearrange(x.squeeze(0), 'c h w -> h w c').detach()
            x = x.numpy()
            plt.imshow(x)
            plt.show()
            display_reverse(images)
            images = []

### Main

In [ ]:
# Define paths
CHECKPOINT_DIR = "/content/drive/MyDrive/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
num_epochs = 1

print(f"Training for only {num_epochs} epochs!")

def main():
    train(
        data_dir="/content/data/healthy",
        batch_size=1,
        checkpoint_path=None,
        lr=2e-5,
        num_epochs=num_epochs
    )
    # inference(os.path.join(CHECKPOINT_DIR, "ddpm_checkpoint"))

if __name__ == '__main__':
    main()

Training for only 1 epochs!
[Log] Random seed set to: 3828849926
[Log] Loading dataset...
[Log] Dataset length: 5680
[Log] First batch shape: torch.Size([1, 4, 128, 128])
[Log] Starting training for 1 epochs...
[Log] Starting epoch 1/1...



----------------------------------------------------------------
Epoch 1/1
:   0%|          | 0/5680 [00:00<?, ?it/s]

[Log] 
Processing batch 1/5680...
[Log] Batch shape: torch.Size([1, 4, 128, 128])
[Log] Loss: 1.0152230262756348



----------------------------------------------------------------
Epoch 1/1
:   0%|          | 1/5680 [00:23<36:54:05, 23.39s/it]

[Log] 
Processing batch 2/5680...
[Log] Batch shape: torch.Size([1, 4, 128, 128])
[Log] Loss: 1.0551211833953857


...

[Log] 
Processing batch 689/5680...
[Log] Batch shape: torch.Size([1, 4, 128, 128])
[Log] Loss: 0.12357264757156372



----------------------------------------------------------------
Epoch 1/1
:  12%|█▏        | 689/5680 [4:10:16<30:28:29, 21.98s/it]

[Log] 
Processing batch 690/5680...
[Log] Batch shape: torch.Size([1, 4, 128, 128])
[Log] Loss: 0.18529285490512848



----------------------------------------------------------------
Epoch 1/1
:  12%|█▏        | 690/5680 [4:10:38<30:11:18, 21.78s/it]

[Log] 
Processing batch 691/5680...
[Log] Batch shape: torch.Size([1, 4, 128, 128])


### Debug

In [19]:
# Debug dataset loading
train_dataset = BRATSDataset(directory="/content/data/healthy")
print("[DEBUG] Dataset length:", len(train_dataset))

# Try loading a single batch
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True, num_workers=0)  # Reduce num_workers
x = next(iter(train_loader))  # Try fetching one batch
x = x.to(device).float()  # Move batch to GPU
print("[DEBUG] First batch loaded:", x.shape)

[DEBUG] Dataset length: 5680
[DEBUG] First batch loaded: torch.Size([2, 4, 128, 128])


In [20]:
scheduler = DDPM_Scheduler(num_time_steps=1000)

# Move model to GPU
model = UNET(
          input_channels=4,
          output_channels=4,
          Channels=[64, 128, 256, 512, 512, 384]
      ).to(device)  # Move model to GPU

print("[DEBUG] Model initialized successfully.")

[DEBUG] Model initialized successfully.


In [21]:
# Load a batch and move to GPU
x = next(iter(train_loader)).to(device).float()
t = torch.randint(0, 500, (x.shape[0],)).to(device)

print("[DEBUG] Running model forward pass...")
# output = model(x, t)  # Forward pass
print("[DEBUG] Forward pass successful, output shape:", output.shape)

[DEBUG] Running model forward pass...
[DEBUG] Forward pass successful, output shape: torch.Size([2, 4, 128, 128])
